In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
import math
import sklearn.preprocessing as sk
import seaborn as sns
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
#import utils
#from myutils import AllTripletSelector,HardestNegativeTripletSelector, RandomNegativeTripletSelector, SemihardNegativeTripletSelector # Strategies for selecting triplets within a minibatch
#from mymetrics import AverageNonzeroTripletsMetric
#from utils import RandomNegativeTripletSelector

from torch.utils.data.sampler import WeightedRandomSampler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import random
from random import randint
from sklearn.model_selection import StratifiedKFold,RepeatedKFold
from sklearn.metrics import roc_curve, auc
from scipy import interp

import numpy as np
import warnings

import math
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn import preprocessing
warnings.filterwarnings("ignore")
#save_results_to = '/home/winston/MOLI/res/cnv_6index/nofs/'
save_results_to = 'F:/MOLI/result/all/'
#torch.manual_seed(42)
random.seed(42)

max_iter = 1

In [6]:
TCGAEx = pd.read_csv("/content/gdrive/MyDrive/multiomics/train_label_lumA_B.csv",index_col=0,header=None)


In [7]:
TCGAEx

,1,2,3,4,5,6,7,8,9,10,...,262,263,264,265,266,267,268,269,270,271
0,,,,,,,,,,,,,,,,,,,,,
0,0.259998,-0.198391,0.876641,-0.208292,1.181464,-0.262674,0.806192,0.704055,0.253328,0.065492,...,1.020069,-0.068761,1.305629,0.185756,-0.858318,-0.408063,-0.101395,-0.266507,1.001205,2
1,-0.343497,0.026803,-2.529036,0.084974,0.714068,0.298348,1.031866,-0.166831,-0.130502,-0.833841,...,0.381677,0.235017,-0.665348,0.188252,-0.948427,-0.119039,-0.095491,0.268777,-0.005900,2
2,0.103036,0.755536,-3.653081,-0.404221,0.088726,-0.658951,1.534416,0.521338,0.883308,-0.163684,...,0.497179,0.008024,-0.636814,-0.451278,-0.475830,0.136512,-0.100465,-0.434367,0.019610,2
3,1.498027,-0.061896,-1.919339,-0.770706,0.979637,-0.351914,0.837508,0.935561,1.755499,0.704042,...,-0.372977,-0.533225,-0.229222,-0.291179,-0.255526,1.740062,-0.101395,0.095526,-0.279280,2
4,0.070095,-0.320080,-2.121374,-0.382156,1.939847,-0.778702,0.565604,-0.613132,-0.023827,0.580460,...,1.440211,0.008396,1.164848,0.085460,-0.765216,0.445302,0.159718,-0.200552,-0.951862,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,-0.506222,0.029083,0.719312,-0.331671,1.007030,0.381355,0.265299,0.227643,-0.525907,1.127026,...,-0.075606,2.075580,-0.446069,-0.539184,-0.459948,-0.072967,-0.101254,-0.448645,0.005884,3
565,-0.912502,0.653279,-0.032195,0.285462,0.841146,0.511421,-0.494159,0.487908,-0.678907,-0.894618,...,0.297026,0.758870,0.190757,1.148564,-0.688770,-0.223543,1.271436,0.102271,-0.216792,3
566,-0.447545,2.303694,0.166780,0.313000,1.046926,0.513491,-0.106066,-0.075666,-0.256510,-0.662691,...,-0.247580,0.086168,0.274922,-0.322638,-0.898913,-0.351916,-0.099888,-0.443525,-0.572310,3


# New Section

In [43]:
!python /content/gdrive/MyDrive/multiomics/lumA_omics_.py

AB*************************************************************
[[-0.27258544  0.38204053 -1.63883197 ... -0.46189481 -0.62808276
   1.        ]
 [ 0.08109363  0.15947801 -0.16257938 ... -0.35104829 -0.86681955
   1.        ]
 [-0.93800161  0.7804159  -0.52237002 ... -0.4508472  -0.2714216
   1.        ]
 ...
 [-1.10385176 -0.32149224  0.93209044 ... -0.38539334  1.40849668
   0.        ]
 [ 0.23859609 -0.46274439 -3.19136252 ...  0.58690334  0.49204
   0.        ]
 [-0.00583795  0.38398572  0.96361109 ... -0.46836018 -0.8497103
   0.        ]]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0

In [ ]:
import pandas as pd